## **Background**
Gcoffee is production house of coffe that selling any coffee and non-coffee menus in several ofline store. In this project, we will analyze sales performance from each store in Q2 period(April-June). 

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

## **Import Datset**

In [22]:
item= r'C:\Users\fahru\Documents\GitHub\GCoffee-Shop-Analysis\Dataset\items25.csv'
tr= r'C:\Users\fahru\Documents\GitHub\GCoffee-Shop-Analysis\Dataset\trx25.csv'


item25= pd.read_csv(item)
trx25= pd.read_csv(tr)
trx25.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   transaction_id     400 non-null    object 
 1   store_id           400 non-null    int64  
 2   payment_method_id  400 non-null    int64  
 3   voucher_id         5 non-null      float64
 4   user_id            397 non-null    float64
 5   original_amount    400 non-null    float64
 6   discount_applied   400 non-null    float64
 7   final_amount       400 non-null    float64
 8   created_at         400 non-null    object 
dtypes: float64(5), int64(2), object(2)
memory usage: 28.3+ KB


## **Data Cleaning**

Select transaction from Store 6 Only.

In [23]:
trx= trx25[(trx25['store_id']==6) & (trx25['user_id'] is not np.nan)]

trx.head()

,transaction_id,store_id,payment_method_id,voucher_id,user_id,original_amount,discount_applied,final_amount,created_at
16,000092da-9845-49bc-8ff6-fa2657b95261,6,1,NaN,1158077.0,18.0,0.0,18.0,2025-06-26 09:20:02
25,0000c107-2d7d-4a83-a856-e68c14280df0,6,3,NaN,2180411.0,21.5,0.0,21.5,2025-06-28 11:32:45
37,00010fc2-3dbc-4aff-9c8d-d735ee2bb93f,6,3,NaN,1995324.0,65.0,0.0,65.0,2025-05-28 11:04:27
43,0001bb6b-12db-4da8-a66d-9cb43e280342,6,4,NaN,1885869.0,48.0,0.0,48.0,2025-05-09 14:35:29
50,0001f12d-84bd-4279-8284-ac157369377c,6,1,NaN,1890292.0,12.0,0.0,12.0,2025-05-10 12:49:46


In [4]:
trx25.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   transaction_id     400 non-null    object 
 1   store_id           400 non-null    int64  
 2   payment_method_id  400 non-null    int64  
 3   voucher_id         5 non-null      float64
 4   user_id            397 non-null    float64
 5   original_amount    400 non-null    float64
 6   discount_applied   400 non-null    float64
 7   final_amount       400 non-null    float64
 8   created_at         400 non-null    object 
dtypes: float64(5), int64(2), object(2)
memory usage: 28.3+ KB


Unfortunetly we can't analyze non-member cust because there are'nt unique value that can represented them. So we will looking for retention rate of member cust that able to see from user_id in their transaction.

created_at must be datetime value, so we need to convert them first.

In [25]:
trx['created_at']= pd.to_datetime(trx['created_at'])
trx.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36 entries, 16 to 392
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   transaction_id     36 non-null     object        
 1   store_id           36 non-null     int64         
 2   payment_method_id  36 non-null     int64         
 3   voucher_id         0 non-null      float64       
 4   user_id            36 non-null     float64       
 5   original_amount    36 non-null     float64       
 6   discount_applied   36 non-null     float64       
 7   final_amount       36 non-null     float64       
 8   created_at         36 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(5), int64(2), object(1)
memory usage: 2.8+ KB


C:\Users\fahru\AppData\Local\Temp\ipykernel_28088\2337533780.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trx['created_at']= pd.to_datetime(trx['created_at'])


## **Analysis**

We trying to analyze retention rate per weeks.

In [27]:
trx['weeks']= trx['created_at'].dt.isocalendar().week
trx.head()

C:\Users\fahru\AppData\Local\Temp\ipykernel_28088\4057075295.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trx['weeks']= trx['created_at'].dt.isocalendar().week


,transaction_id,store_id,payment_method_id,voucher_id,user_id,original_amount,discount_applied,final_amount,created_at,weeks
16,000092da-9845-49bc-8ff6-fa2657b95261,6,1,NaN,1158077.0,18.0,0.0,18.0,2025-06-26 09:20:02,26
25,0000c107-2d7d-4a83-a856-e68c14280df0,6,3,NaN,2180411.0,21.5,0.0,21.5,2025-06-28 11:32:45,26
37,00010fc2-3dbc-4aff-9c8d-d735ee2bb93f,6,3,NaN,1995324.0,65.0,0.0,65.0,2025-05-28 11:04:27,22
43,0001bb6b-12db-4da8-a66d-9cb43e280342,6,4,NaN,1885869.0,48.0,0.0,48.0,2025-05-09 14:35:29,19
50,0001f12d-84bd-4279-8284-ac157369377c,6,1,NaN,1890292.0,12.0,0.0,12.0,2025-05-10 12:49:46,19


In [34]:
np.sort(trx['weeks'].unique().tolist())

array([14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26])

In [39]:
first= trx.groupby('user_id')['weeks'].min().reset_index()
first.columns= ['user_id','first_week']
trx= trx.merge(first, on='user_id', how='left')
trx.head()

,transaction_id,store_id,payment_method_id,voucher_id,user_id,original_amount,discount_applied,final_amount,created_at,weeks,first_week
0,000092da-9845-49bc-8ff6-fa2657b95261,6,1,NaN,1158077.0,18.0,0.0,18.0,2025-06-26 09:20:02,26,26
1,0000c107-2d7d-4a83-a856-e68c14280df0,6,3,NaN,2180411.0,21.5,0.0,21.5,2025-06-28 11:32:45,26,26
2,00010fc2-3dbc-4aff-9c8d-d735ee2bb93f,6,3,NaN,1995324.0,65.0,0.0,65.0,2025-05-28 11:04:27,22,22
3,0001bb6b-12db-4da8-a66d-9cb43e280342,6,4,NaN,1885869.0,48.0,0.0,48.0,2025-05-09 14:35:29,19,19
4,0001f12d-84bd-4279-8284-ac157369377c,6,1,NaN,1890292.0,12.0,0.0,12.0,2025-05-10 12:49:46,19,19


In [44]:
# Seacrh cohort distance
from operator import attrgetter

trx['cohort_distnace']= (trx['weeks']-trx['first_week']).apply(lambda x:x+1)
trx.head()

,transaction_id,store_id,payment_method_id,voucher_id,user_id,original_amount,discount_applied,final_amount,created_at,weeks,first_week,cohort_distnace
0,000092da-9845-49bc-8ff6-fa2657b95261,6,1,NaN,1158077.0,18.0,0.0,18.0,2025-06-26 09:20:02,26,26,1
1,0000c107-2d7d-4a83-a856-e68c14280df0,6,3,NaN,2180411.0,21.5,0.0,21.5,2025-06-28 11:32:45,26,26,1
2,00010fc2-3dbc-4aff-9c8d-d735ee2bb93f,6,3,NaN,1995324.0,65.0,0.0,65.0,2025-05-28 11:04:27,22,22,1
3,0001bb6b-12db-4da8-a66d-9cb43e280342,6,4,NaN,1885869.0,48.0,0.0,48.0,2025-05-09 14:35:29,19,19,1
4,0001f12d-84bd-4279-8284-ac157369377c,6,1,NaN,1890292.0,12.0,0.0,12.0,2025-05-10 12:49:46,19,19,1


In [47]:
trx[trx['user_id']==1158077.0]

,transaction_id,store_id,payment_method_id,voucher_id,user_id,original_amount,discount_applied,final_amount,created_at,weeks,first_week,cohort_distnace
0,000092da-9845-49bc-8ff6-fa2657b95261,6,1,NaN,1158077.0,18.0,0.0,18.0,2025-06-26 09:20:02,26,26,1
